## Part 1

In [27]:
import time
with open('input.txt') as f:
    lines = f.read().splitlines()
start_time = time.perf_counter_ns()

labels = ['A','K','Q','J','T','9','8','7','6','5','4','3','2']
types = ["Five of a kind", "Four of a kind", "Full house", "Three of a kind", "Two pair", "One pair", "High card"]

class Hand:
    def __init__(self, inp):
        self.label = inp[0]
        self.bid = inp[1]
        self.type = self.type_finder()
    
    def type_finder(self):
        # Five of a kind - check if all chars are the same
        if len(set(self.label)) == 1:
            return "Five of a kind"
        # Four of a kind - check if any char is repeated 4 times
        elif 4 in [self.label.count(label) for label in labels]:
            return "Four of a kind"
        # Full house - check if any char is repeated 3 times and another char is repeated 2 times
        elif 3 in [self.label.count(label) for label in labels] and 2 in [self.label.count(label) for label in labels]:
            return "Full house"
        # Three of a kind - check if any char is repeated 3 times but the 2 other chars are not repeated
        elif 3 in [self.label.count(label) for label in labels] and 2 not in [self.label.count(label) for label in labels]:
            return "Three of a kind"
        # Two pair - check if any char is repeated 2 times and another char is repeated 2 times
        elif [self.label.count(label) for label in labels].count(2) == 2:
            return "Two pair"
        # One pair - check if any char is repeated 2 times but the 3 other chars are not repeated
        elif 2 in [self.label.count(label) for label in labels] and 3 not in [self.label.count(label) for label in labels]:
            return "One pair"
        # High card - check if all chars are different
        elif len(set(self.label)) == 5:
            return "High card"
        
        # Error    
        else:
            return "Error"

    # method to compare hands
    def __lt__(self, other):
        # if the type is the same, compare the first char, if they are the same, compare the next one and so on, if they are all the same, return true
        if self.type == other.type:
            for i in range(0,5):
                if labels.index(self.label[i]) != labels.index(other.label[i]):
                    return labels.index(self.label[i]) < labels.index(other.label[i])
            return True
        # if the type is different, compare the type
        else:
            return types.index(self.type) < types.index(other.type)

    def __repr__(self):
        return f"{self.label} {self.bid} {self.type}"

# dealing with input
hands = []
for line in lines:
    hand = Hand(line.split())
    hands.append(hand)

# create a dictionary with the types as keys and the hands as values
types = {'Five of a kind': [], 'Four of a kind': [], 'Full house': [], 'Three of a kind': [], 'Two pair': [], 'One pair': [], 'High card': []}
for hand in hands:
    types[hand.type].append(hand)

# sort the hands in each type by the label using the __lt__ method
for key in types:
    types[key].sort(reverse=True)
    
# add all hands in types to list hands, sorted by type 'High card' first
hands = []
for key in reversed(types):
    hands += types[key]   

# for each hand, multiply the index+1 by the bid and add to the total
total = 0
for i in range(0,len(hands)):
    total += (i+1)*int(hands[i].bid)
print(total)
print(f"Time taken: {round((time.perf_counter_ns() - start_time)/1000000, 3)}ms")

248812215
Time taken: 10.459ms


## Part 2

In [36]:
import time
with open('input.txt') as f:
    lines = f.read().splitlines()
start_time = time.perf_counter_ns()

labels = ['A','K','Q','T','9','8','7','6','5','4','3','2', 'J']
types = ["Five of a kind", "Four of a kind", "Full house", "Three of a kind", "Two pair", "One pair", "High card"]

class Hand:
    def __init__(self, inp):
        self.label = inp[0]
        self.bid = inp[1]
        self.type = self.type_finder()
    
    #def type_finder(self):
        ## Five of a kind - check if all chars are the same
        #if len(set(self.label)) == 1:
        #    return "Five of a kind"
        ## Four of a kind - check if any char is repeated 4 times
        #elif 4 in [self.label.count(label) for label in labels]:
        #    return "Four of a kind"
        ## Full house - check if any char is repeated 3 times and another char is repeated 2 times
        #elif 3 in [self.label.count(label) for label in labels] and 2 in [self.label.count(label) for label in labels]:
        #    return "Full house"
        ## Three of a kind - check if any char is repeated 3 times but the 2 other chars are not repeated
        #elif 3 in [self.label.count(label) for label in labels] and 2 not in [self.label.count(label) for label in labels]:
        #    return "Three of a kind"
        ## Two pair - check if any char is repeated 2 times and another char is repeated 2 times
        #elif [self.label.count(label) for label in labels].count(2) == 2:
        #    return "Two pair"
        ## One pair - check if any char is repeated 2 times but the 3 other chars are not repeated
        #elif 2 in [self.label.count(label) for label in labels] and 3 not in [self.label.count(label) for label in labels]:
        #    return "One pair"
        ## High card - check if all chars are different
        #elif len(set(self.label)) == 5:
        #    return "High card"
        #
        ## Error    
        #else:
        #    return "Error"
        
    # We now have jokers, represented by J, they can represent any card
    def type_finder(self):
        label_counts = [self.label.count(label) for label in labels]
        if 'J' in self.label:
            joker_count = self.label.count('J')
            non_joker_counts = [count for label, count in zip(labels, label_counts) if label != 'J']
            max_non_joker_count = max(non_joker_counts) if non_joker_counts else 0

            # Try to form the best possible hand by replacing the jokers
            for replacement_count in range(5, max_non_joker_count, -1):
                if max_non_joker_count + joker_count >= replacement_count:
                    if replacement_count == 5:
                        return "Five of a kind"
                    elif replacement_count == 4:
                        return "Four of a kind"
                    elif replacement_count == 3 and (2 in [count for count in non_joker_counts if count != max_non_joker_count] or joker_count > 1):
                        return "Full house"
                    elif replacement_count == 3:
                        return "Three of a kind"
                    elif replacement_count == 2 and (non_joker_counts.count(2) > 0 or non_joker_counts.count(1) > 1 or joker_count > 1):
                        return "Two pair"
                    elif replacement_count == 2:
                        return "One pair"
            return "High card"
        else:
            if 5 in label_counts:
                return "Five of a kind"
            elif 4 in label_counts:
                return "Four of a kind"
            elif 3 in label_counts and 2 in [count for count in label_counts if count != 3]:
                return "Full house"
            elif 3 in label_counts:
                return "Three of a kind"
            elif label_counts.count(2) == 2:
                return "Two pair"
            elif 2 in label_counts:
                return "One pair"
            else:
                return "High card"

    # method to compare hands
    def __lt__(self, other):
        # if the type is the same, compare the first char, if they are the same, compare the next one and so on, if they are all the same, return true
        if self.type == other.type:
            for i in range(0,5):
                if labels.index(self.label[i]) != labels.index(other.label[i]):
                    return labels.index(self.label[i]) < labels.index(other.label[i])
            return True
        # if the type is different, compare the type
        else:
            return types.index(self.type) < types.index(other.type)

    def __repr__(self):
        return f"{self.label} {self.bid} {self.type}"

# dealing with input
hands = []
for line in lines:
    hand = Hand(line.split())
    print(hand)
    hands.append(hand)

# create a dictionary with the types as keys and the hands as values
types = {'Five of a kind': [], 'Four of a kind': [], 'Full house': [], 'Three of a kind': [], 'Two pair': [], 'One pair': [], 'High card': []}
for hand in hands:
    types[hand.type].append(hand)

# sort the hands in each type by the label using the __lt__ method
for key in types:
    types[key].sort(reverse=True)
    
# add all hands in types to list hands, sorted by type 'High card' first
hands = []
for key in reversed(types):
    hands += types[key]   

# for each hand, multiply the index+1 by the bid and add to the total
total = 0
for i in range(0,len(hands)):
    total += (i+1)*int(hands[i].bid)
print(total)
print(f"Time taken: {round((time.perf_counter_ns() - start_time)/1000000, 3)}ms")

TQ5TT 421 Three of a kind
65K2J 973 Two pair
K543T 50 High card
AT68Q 952 High card
JKTTJ 766 Four of a kind
55Q55 860 Four of a kind
J9427 977 Two pair
K36QA 569 High card
TQQJ2 710 Three of a kind
KK274 900 One pair
78667 841 Two pair
3J3A3 753 Four of a kind
35333 832 Four of a kind
4T4TT 526 Full house
5JJ38 283 Full house
3J999 48 Four of a kind
94549 220 Two pair
TJ777 917 Four of a kind
8483J 509 Three of a kind
J6628 103 Three of a kind
Q7999 423 Three of a kind
6AQTT 711 One pair
Q77Q7 914 Full house
4K44T 898 Three of a kind
J87A7 793 Three of a kind
KKK55 750 Full house
KK336 142 Two pair
T6JJT 410 Four of a kind
58Q44 313 One pair
Q8959 41 One pair
89989 224 Full house
27598 271 High card
KKJ77 948 Three of a kind
27544 751 One pair
239TA 384 High card
7396K 937 High card
J7JQT 493 Full house
29Q3K 735 High card
83645 367 High card
755AA 112 Two pair
J43K5 465 Two pair
Q6T85 830 High card
Q7J35 360 Two pair
28877 58 Two pair
6TJ54 173 Two pair
7Q4KQ 969 One pair
8AA99 70 Tw